In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from collections import Counter

import re

,Unnamed: 0,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_age_at_time_of_event,verslagen_report_specialism,verslagen_report_start_date,verslagen_report_content_cleaned
0,0,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Telefonisch consult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,66,"Maag-, Darm- en Leverziekten",2019-11-25 12:45:00,samenvatting chronische hbv maart sec opinion...
1,1,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VDER-[LASTNAME]...,66,"Maag-, Darm- en Leverziekten",2019-11-08 00:00:00,patient lastname vderlastname vanbt birthdate ...
2,2,0001A243648CFB955C3718FD1CE81276D1F4F260,Poliklinische Brief,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",65,"Maag-, Darm- en Leverziekten",2019-05-09 15:45:00,dhr ap nederstigt huisarts berg b cc city da...
3,3,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,65,"Maag-, Darm- en Leverziekten",2019-05-09 15:38:00,samenvatting chronische hbv maart sec opinion...
4,4,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VAN DER-[LASTNA...,65,"Maag-, Darm- en Leverziekten",2019-03-26 00:00:00,patient lastname derlastname vanbt birthdate v...
...,...,...,...,...,...,...,...,...
127691,127691,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Consult,Samenvatting: \n1e consult\r\n-Type 1e consult...,83,Interne Geneeskunde,2015-03-20 08:13:00,samenvatting e consult type e consult uitgebre...
127692,127692,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, Kliniek: vervolgconsult",Samenvatting: \nDecursus\r\n-Type decursus: De...,83,Interne Geneeskunde,2015-01-14 15:39:00,samenvatting decursus type decursus decursus s...
127693,127693,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, SEH",Samenvatting: \nVerpleegkundige verslaglegging...,83,Spoedeisende Hulp,2014-12-21 09:31:00,samenvatting verpleegkundige verslaglegging ve...
127694,127694,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, SEH",Samenvatting: \nMedisch Dossier\r\n[ Vk Sputov...,79,Spoedeisende Hulp,2010-11-10 21:03:00,samenvatting medisch dossier vk sputovamo leef...


In [2]:
df_verslagen = pd.read_csv('a:/bloeding-met-patientenlijst-2/bloeding-met-patientenlijst-4-verslagen.csv')
df_verslagen

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,2020-11-26 09:53:00
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00
...,...,...,...,...
9572,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Consult,Samenvatting: \n1e consult\r\n-Type 1e consult...,2015-03-20 08:13:00
9573,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, Kliniek: vervolgconsult",Samenvatting: \nDecursus\r\n-Type decursus: De...,2015-01-14 15:39:00
9574,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, SEH",Samenvatting: \nVerpleegkundige verslaglegging...,2014-12-21 09:31:00
9575,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, SEH",Samenvatting: \nMedisch Dossier\r\n[ Vk Sputov...,2010-11-10 21:03:00
